<a href="https://colab.research.google.com/github/forthenergy/assingment/blob/main/titanic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 타이타닉 생존자 예측 분석 - 실습

이 노트북은 Kaggle의 Titanic 데이터셋을 사용하여 생존자를 예측하는 모델을 개발합니다.
빈칸을 채워 모델의 성능을 향상시켜보세요!

## 목차
1. 데이터 로드 및 기본 정보 확인
2. 데이터 전처리
3. 모델 학습 및 평가
4. 예측 및 제출
5. 하이퍼파라미터 튜닝

## 1. 데이터 로드 및 기본 정보 확인

이 섹션에서는 다음 작업들을 수행합니다:
- 필요한 라이브러리 임포트
- train.csv와 test.csv 파일 로드
- 데이터의 기본 구조 확인

### 학습 포인트
- Pandas를 사용한 CSV 파일 읽기
- 데이터프레임의 기본 구조 이해
- 데이터 탐색의 중요성

In [22]:
pip install tensorflow_decision_forests

In [23]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.11.0


In [24]:
# TODO: train.csv와 test.csv 파일을 읽어오세요
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 데이터 확인
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## 2. 데이터 전처리

이 섹션에서는 모델 학습을 위한 데이터 전처리를 수행합니다:
- 결측치 처리
- 특수문자 제거 및 텍스트 정규화
- 티켓 정보 분리 및 가공

### 학습 포인트
- 데이터 전처리의 중요성 이해
- 문자열 처리 방법 학습
- Feature Engineering의 기본 개념
- 데이터프레임 조작 방법

In [25]:
# TODO: 데이터 전처리 함수를 완성하세요
def preprocess(df):
    df = df.copy()

    def normalize_name(x):
        # 이름에서 특수문자를 제거하고 정규화하는 코드를 작성하세요
        """이름을 보면 Cumings, Mrs. John Bradley (Florence Briggs Thayer)이라는 데이터가 입력되어있다.
        Cumings은 성(last name)이다
        Mrs(부인)인데 John Bradley(남자이름)이다. 그 당시 부인이 남편의 이름을 썼다고 한다.
        이에 괄호 안에 있는 것은 혼전이름이다. 해당 데이터의 혼전이름은 Florence Briggs Thayer인 것이다."""
        ## 토큰화 하라는 문제를 보고 코드를 변경
        ### 1, 이름에서 ()특수문자를 제거
        ### 2. 공백처리
        ### 3. 모두 소문자화
        import re
        x = re.sub(r"[^\w\s]", "", x)
        x = re.sub(r"\s+", " ", x).strip()
        x = x.lower()
        return x

    def ticket_number(x):
        # 티켓 번호만 추출하는 코드를 작성하세요
        import re
        match = re.search(r"(\d+)$", x)
        return match.group(1) if match else None

    def ticket_item(x):
        # 티켓 종류를 추출하는 코드를 작성하세요
        # ex) A/5 21171 에서 A/5를 추출
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        else:
            return items[0]

    # 전처리 함수들을 데이터프레임에 적용하세요
    df["Name"] = df['Name'].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)

    return df

In [26]:
# 전처리 함수 적용
preprocessed_train_df = preprocess(train_df)
preprocessed_test_df = preprocess(test_df)

preprocessed_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,braund mr owen harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,cumings mrs john bradley florence briggs thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,heikkinen miss laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,futrelle mrs jacques heath lily may peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,allen mr william henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [27]:
# TODO: 모델에 사용할 특성(features)을 선정하세요
# 제외해야 할 컬럼: "Ticket", "PassengerId", "Survived"
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


In [28]:
# TODO: 이름을 토큰화하는 함수를 완성하세요
def tokenize_names(features, labels=None):
    features["Name"] = tf.strings.split(features["Name"])
    return features, labels
    ## 텐서플로우에 for문(리스트 컴프리헨션)을 쓰지 못하는것 같다.
    ## tf.strings.split로 대체함

# TensorFlow Dataset 생성
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df, label="Survived").map(tokenize_names)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_test_df).map(tokenize_names)

## 3. 모델 학습 및 평가

이 섹션에서는 기본 모델을 구성하고 학습시킵니다:
- GradientBoostedTrees 모델 설정
- 기본 파라미터로 모델 학습
- 모델 성능 평가

### 학습 포인트
- 의사결정나무 기반 모델의 이해
- 모델 파라미터의 의미와 영향
- 모델 평가 지표의 이해

In [61]:
# TODO: GradientBoostedTreesModel의 파라미터를 설정하세요
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    random_seed=1234,
    num_trees=300,
    max_depth=3,
    min_examples=1,
    shrinkage=0.7
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Accuracy: 0.8478260636329651 Loss:0.8092525601387024


## 4. 예측 및 제출

이 섹션에서는 학습된 모델을 사용하여 예측을 수행합니다:
- 테스트 데이터에 대한 예측 수행
- Kaggle 제출 형식에 맞게 결과 저장
- 제출 파일 생성

### 학습 포인트
- 모델을 사용한 예측 방법
- 예측 결과의 후처리
- Kaggle 제출 형식 이해

In [63]:
# Kaggle 제출용 예측 결과 생성 함수
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(test_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

# TODO: threshold 값을 적절히 설정하세요
predictions = prediction_to_kaggle_format(model, threshold=0.45)
predictions.to_csv("base_submission.csv", index=False)
print("Base model submission file created!")

Base model submission file created!


## 5. 하이퍼파라미터 튜닝

모델의 성능을 향상시키기 위해 하이퍼파라미터 튜닝을 진행합니다.

In [65]:
# TODO: RandomSearch 튜너의 하이퍼파라미터 탐색 범위를 설정하세요
tuner = tfdf.tuner.RandomSearch(num_trials=20)

# 기본 파라미터
tuner.choice("min_examples", [2, 5, 10, 20, 50])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

# LOCAL 전략 파라미터
local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [4, 5, 6, 7])

# GLOBAL 전략 파라미터
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128])

# TODO: 추가 하이퍼파라미터를 설정하세요
tuner.choice("shrinkage", [0.01, 0.05, 0.1, 0.2, 0.3])
tuner.choice("num_candidate_attributes_ratio", [0.1, 0.2, 0.5, 1.0])

# 튜닝된 모델 학습
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Tuned model - Accuracy: {tuned_self_evaluation.accuracy} Loss: {tuned_self_evaluation.loss}")

Use /tmp/tmp4jwvaj23 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.610815. Found 891 examples.
Training model...
Model trained in 0:00:03.160203
Compiling model...
Model compiled.
Tuned model - Accuracy: 0.8493150472640991 Loss: 0.6972012519836426


In [67]:
# TODO: 앙상블 모델의 수와 파라미터를 설정하세요
predictions = None
num_predictions = 0

for i in range(100):  # 앙상블할 모델 수 설정
    print(f"Training model {i+1}/100")
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0,
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True,
        # 추가 파라미터를 설정하세요
        max_depth=int(np.random.choice([4, 5, 6, 7])),
        min_examples=int(np.random.choice([2, 5, 10, 20, 50])),
        shrinkage=np.random.choice([0.01, 0.05, 0.1, 0.2, 0.3]),
        random_seed=i
    )
    model.fit(train_ds)

    sub_predictions = model.predict(test_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

# 앙상블 예측 평균 계산
predictions /= num_predictions

# Kaggle 제출용 파일 생성
ensemble_predictions = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": (predictions >= 0.5).astype(int)
})

ensemble_predictions.to_csv("submission.csv", index=False)
print("Submission file created!")

Training model 1/100
Training model 2/100
Training model 3/100
Training model 4/100
Training model 5/100
Training model 6/100
Training model 7/100
Training model 8/100
Training model 9/100
Training model 10/100
Training model 11/100
Training model 12/100
Training model 13/100
Training model 14/100
Training model 15/100
Training model 16/100
Training model 17/100
Training model 18/100
Training model 19/100
Training model 20/100
Training model 21/100
Training model 22/100
Training model 23/100
Training model 24/100
Training model 25/100
Training model 26/100
Training model 27/100
Training model 28/100
Training model 29/100
Training model 30/100
Training model 31/100
Training model 32/100
Training model 33/100
Training model 34/100
Training model 35/100
Training model 36/100
Training model 37/100
Training model 38/100
Training model 39/100
Training model 40/100
Training model 41/100
Training model 42/100
Training model 43/100
Training model 44/100
Training model 45/100
Training model 46/1